In [ ]:
#Do a super resolution with ESRGAN
import cv2
import numpy as np
from ProSR import ProSR

# Load the ProSR model
model = ProSR()

# Open the low resolution video
cap = cv2.VideoCapture("low_resolution_video.avi")

# Get the video dimensions
width = int(cap.get(3))
height = int(cap.get(4))

# Define the codec and create a video writer object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('high_resolution_video.avi',fourcc, 30.0, (width*4, height*4))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        # Perform super-resolution on the frame
        sr_frame = model.predict(frame, scale=4)

        # Write the high resolution frame to the output video
        out.write(sr_frame)

        # Display the frame
        cv2.imshow('frame',sr_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the video writer and capture objects
cap.release()
out.release()

# Close all windows
cv2.destroyAllWindows()


In [ ]:
#COLORIZATION
from skimage.io import imread, imshow
from skimage.color import rgb2hsv
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from skimage import filters
import skimage
from PIL import Image
import os
warnings.filterwarnings("ignore")
video = cv2.VideoCapture(r"C:\Users\tzhou\Downloads\50x50 micrometer channel D 0.075ul per min 40x with dextran 02(1).mp4")

i = 0
while(1):
    success, bags = video.read()
    cv2.imwrite("image"+str(i)+"GRAY.png", bags)
    lab = cv2.cvtColor(bags, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)
    limg = cv2.merge((cl,a,b))
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)


    bags_hsv = enhanced_img
   
    

    fig, ax = plt.subplots(1, 3, figsize=(12,4))
    ax[0].imshow(bags_hsv[:,:,0], cmap='gray')
    ax[0].set_title('Hue')
    ax[1].imshow(bags_hsv[:,:,1], cmap='gray')
    ax[1].set_title('Saturation')
    ax[2].imshow(bags_hsv[:,:,2], cmap='gray')
    ax[2].set_title('Value');


    fig, ax = plt.subplots(1, 1, figsize=(15, 5))
    ax[0].imshow(bags_hsv[:,:,0],cmap='hsv')
    ax[0].set_title('hue')
    ax[1].imshow(bags_hsv[:,:,1],cmap='hsv')
    ax[1].set_title('transparency')
    ax[2].imshow(bags_hsv[:,:,2],cmap='hsv')
    ax[2].set_title('value')
    fig.colorbar(imshow(bags_hsv[:,:,0],cmap='hsv'))
    fig.tight_layout()

    # refer to hue channel (in the colorbar)
    lower_mask = bags_hsv[:, :, 0] > 110

    # refer to hue channel (in the colorbar)
    upper_mask = bags_hsv[:, :, 0] < 140

    # refer to transparency channel (in the colorbar)
    saturation_mask = bags_hsv[:, :, 1] <115

    mask = upper_mask * lower_mask * saturation_mask
    red = bags[:, :, 0] * mask
    green = bags[:, :, 1] * mask
    blue = bags[:, :, 2] * mask
    bags_masked = np.dstack((red, green, blue))
    imgplot = plt.imshow(bags_masked)

    plt.savefig('image' + str(i)+ '.png')
    i+=1
video.release()



   

In [ ]:
#turn potentially non RGB frames into RGB for video stitching with av
import av
directory = r"C:\Users\tzhou"
m = 0
container = av.open("output.mp4", "w")

# Set the video format and frame rate
video_stream = container.add_stream("vp9", rate=30)
video_stream.height = 500
video_stream.width = 1500


for file in os.listdir(directory):
    for m in range(i):
        if file == 'image' + str(m) + '.png':
            # Open the image file
            with Image.open(file) as image:
                #image.show()
                if image.mode != "RGB":
                    image = image.convert("RGB")
                    image.save('image' + str(m) + '_rgb.png')

            

In [ ]:
#user select background colors and they get subtracted out
from PIL import Image

# Define the threshold
thresholdB = 255

# Iterate through the pixels

print(i)
for m in range(i):
    im = Image.open("image" + str(m) + "_rgb.png")
    pixels = im.load()
    for x in range(im.width):
        for y in range(im.height):
            r, g, b = pixels[x, y]
            # Check if the pixel is above the threshold
            if b == thresholdB or g == 0:
                # Change the pixel to white
                pixels[x, y] = (0, 0, 0)

    im.save("modified_image" + str(m) + ".jpg")
print(i)

In [ ]:
#100 PERCENT FUNCTIONAL VIDEO STITCHING ALGORITHM COMPLETED!!!!!!!!!!!!!!!

frames = []
for n in range(i):
    frames.append(cv2.imread('modified_image' + str(n) + '.jpg'))

# Set the frame width and height
frame_width = frames[0].shape[1]
frame_height = frames[0].shape[0]

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('OutputNEW3.mp4', fourcc, 30.0, (frame_width,frame_height))

# Write the frames to the video file
for frame in frames:
    out.write(frame)

# Release the VideoWriter object
out.release()

In [ ]:
#ROI SELECTION

import cv2
import numpy as np
canvas = None
drawing = False # true if mouse is pressed

#Retrieve first frame
def initialize_camera(cap):
    _, frame = cap.read()
    return frame


# mouse callback function
def mouse_draw_rect(event,x,y,flags, params):
    global drawing, canvas

    if drawing:
        canvas = params[0].copy()

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        params.append((x,y)) #Save first point

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.rectangle(canvas, params[1],(x,y),(0,255,0),2)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        params.append((x,y)) #Save second point
        cv2.rectangle(canvas,params[1],params[2],(0,255,0),2)


def select_roi(frame):
    global canvas
    canvas = frame.copy()
    params = [frame]
    ROI_SELECTION_WINDOW = 'Select ROI'
    cv2.namedWindow(ROI_SELECTION_WINDOW)
    cv2.setMouseCallback(ROI_SELECTION_WINDOW, mouse_draw_rect, params)
    roi_selected = False
    while True:
        cv2.imshow(ROI_SELECTION_WINDOW, canvas)
        key = cv2.waitKey(10)

        #Press Enter to break the loop
        if key == 13:
            break;


    cv2.destroyWindow(ROI_SELECTION_WINDOW)
    roi_selected = (3 == len(params))

    if roi_selected:
        p1 = params[1]
        p2 = params[2]
        if (p1[0] == p2[0]) and (p1[1] == p2[1]):
            roi_selected = False

    #Use whole frame if ROI has not been selected
    if not roi_selected:
        print('ROI Not Selected. Using Full Frame')
        p1 = (0,0)
        p2 = (frame.shape[1] - 1, frame.shape[0] -1)


    return roi_selected, p1, p2
if __name__ == '__main__':

    cap = cv2.VideoCapture(r"C:\Users\tzhou\OutputNEW1.mp4")
    #Grab first frame
    first_frame = initialize_camera(cap)

    #Select ROI for processing. Hit Enter after drawing the rectangle to finalize selection
    roi_selected, point1, point2 = select_roi(first_frame)    

    #Grab ROI of first frame
    first_frame_roi = first_frame[point1[1]:point2[1], point1[0]:point2[0], :]
    a = (point1[0]) #x upper left
    b = (point1[1]) #y upper left
    c = (point2[0]) #x lower right
    d = (point2[1]) #y lower right

    # Define the region to keep (x1, y1, x2, y2)
    x1, y1, x2, y2 = (a, b, c, d)
    t = 0
    while True:
        ret, frame = cap.read()
        # Create black image with same dimensions as frame
        black_image = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)

        # Copy region of interest from frame to black image
        black_image[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
        image_bgr = cv2.cvtColor(black_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite("image"+str(t)+"ROI.jpg", image_bgr)
        t += 1

In [ ]:
frames = []
for n in range(t):
    frames.append(cv2.imread('image' + str(n) + 'ROI.jpg'))
    
# Set the frame width and height
frame_width = frames[0].shape[1]
frame_height = frames[0].shape[0]

    # Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('OutputNEW0.mp4', fourcc, 30.0, (frame_width,frame_height))

    # Write the frames to the video file
for frame in frames:
    out.write(frame)

    # Release the VideoWriter object
out.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
#CLEAR IMAGES THAT WERE SAVED 
import os

# List all the files in the current working directory
files = os.listdir()

# Loop through the files
for file in files:
    # Check if the file name contains the desired string
    if "image" in file:
        # Delete the file
        os.remove(file)

In [ ]:
#COMPUTE FLOW STATISTIC
import cv2

# Create a VideoCapture object
cap = cv2.VideoCapture(r"C:\Users\tzhou\OutputNEW0.mp4")

# Read the first frame of the video
ret, prev_frame = cap.read()

# Convert the frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Define the parameters for the Farneback method
params = dict(pyr_scale = 0.5, levels = 3, winsize = 15, iterations = 3, poly_n = 5, poly_sigma = 1.2, flags = 0)

# Create an empty list to store the optical flow vectors
flow = []

# Read the frames of the video
while True:
    ret, frame = cap.read()
    # Exit the loop if there are no more frames
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate the optical flow vectors
    flow_vectors = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, **params)

    # Append the flow vectors to the list
    flow.append(flow_vectors)

    # Update the previous frame
    prev_gray = gray

# Release the VideoCapture object
cap.release()

# Calculate the rate of flow
rate_of_flow = []
for i in range(1, len(flow)):
    rate_of_flow.append(np.mean(np.abs(flow[i] - flow[i-1])))
average = np.mean(rate_of_flow)

# Print the rate of flow
print(average)



In [ ]:
import matplotlib.pyplot as plt

# Create data
time = range(1, len(flow))

# Create a new figure and axes
fig, ax = plt.subplots()

# Plot the data
ax.plot(time, rate_of_flow)

# Add labels to the x- and y-axes
ax.set_xlabel('Time (frame number)')
ax.set_ylabel('Rate of Flow (pixels/frame)')

# Add a title to the graph
ax.set_title('Rate of Flow over Time')

# Show the graph
plt.show()
